# Snapping the landscape to Steady State with **variable theta** and **rainfall**

This is added to **linear** precipitation scenarios

In [1]:
# Importing what I need, they are all installable from conda in case you miss one of them
import numpy as np
import xarray as xr
import xsimlab as xs
import fastscape as fst
from fastscape.processes.context import FastscapelibContext
import numba as nb
import math
import zarr
import itertools
import matplotlib.pyplot as plt
# Ignore that last
# %load_ext xsimlab.ipython
from hkb_diamondsquare import DiamondSquare as DS
import lsdtopytools as lsd


import constant_prec_fastscape_ext 
import constant_precipitation
from fastscape.processes.grid import UniformRectilinearGrid2D

## Snapping the landscape to steady state

In [2]:
    
# First creating a process to handle elevation changes (it makes sure fastscape updates topography at each time step, it is a convoluted logic inherited from fortran. Ignore.)
@xs.process
class Elevation(fst.processes.SurfaceTopography):
    elevation = xs.variable(
        dims=('y', 'x'),
        intent='inout',
        description='surface topography elevation'
    )
    
    fs_context = xs.foreign(FastscapelibContext, 'context')
    shape = xs.foreign(fst.processes.UniformRectilinearGrid2D, 'shape')
    
    def finalize_step(self):
        
        self.elevation = self.fs_context["h"].reshape(self.shape)



# This is the internal function actually calculating topography
# Numba makes it fast
# This is the internal function actually calculating topography
# Numba makes it fast
@nb.njit()
def _snap2steadySF_(Z,stack,receivers, lengths,AQ,m,n,K,EU):
    """
    Z -> vectorised elevation
    stack -> Topological order from bottom to top (value = node ID)
    receivers -> node ID to its steepest descent receiver
    length -> distance to the receiver
    AQ - > vectorised area/discharge
    m -> m in SPL
    n -> n in spl
    K -> vectorised erodibility
    EU -> European Union or Erosion/uplift (depending what you need)
    """
    # Navigating stack from bottom to top
    for node in stack:
        # if the receiver is the node itself ignore (is base level by convention)
        if(receivers[node] == node):
            continue
        #else apply Simon's formula (if you want to retrieve it it is actually S = ksn*A^theta solved in the SPL referential with forward difference)
        divterm1 = (K[node] * math.pow(AQ[node], m))
        if(divterm1 > 0):
            powterm = math.pow( EU[node]/divterm1, 1/n) 
        else:
            powterm = 0;
        
            
        Z[node] = Z[receivers[node]] + lengths[node] * powterm


# the process of snapping to steady state, only works with single flow
@xs.process
class Snap2SteadySF:
    #Needed variables
    m = xs.variable(
        dims=(),
        intent='inout',
        description='m exp spl')
    n = xs.variable(
        dims=(),
        intent='inout',
        description='n exp spl')
    K = xs.variable(
        dims=('y','x'),
        intent='inout',
        description='Erodibility coefficient')
    EU = xs.variable(
        dims=('y','x'),
        intent='inout',
        description='Erosion/Uplift field to target')
    
    StStZ = xs.variable(
        dims=('y','x'),
        intent='out',
        description='Erosion/Uplift field to target')
    
    # Foreign variables
    ## Shape of the landscape in ny,nx
    shape = xs.foreign(fst.processes.UniformRectilinearGrid2D, 'shape')
    ## Access to fortran, ignore
    fs_context = xs.foreign(FastscapelibContext, 'context')
    ## Topological order bottom to top
    stack = xs.foreign(fst.processes.FlowRouter, 'stack')
    ## Receiver array nodeID -> receiver Node ID, only one cause SF
    receivers = xs.foreign(fst.processes.FlowRouter, 'receivers')
    ## length array nodeID -> legth to receiver Node ID
    lengths = xs.foreign(fst.processes.FlowRouter, 'lengths')
    ## Flow accumulator
    flowacc = xs.foreign(fst.processes.FlowAccumulator, 'flowacc')
        
        #What happen at run time
    def run_step(self):
        # Getting the elevation from fortran
        VZ = self.fs_context["h"]
        # Calling the numba function making sure we vectorise the thingies that need (.ravel())
        _snap2steadySF_(VZ,self.stack,self.receivers, self.lengths,self.flowacc.ravel(),self.m,self.n,self.K.ravel(),self.EU.ravel())
        # Back transmitting elev data
        self.StStZ = VZ.reshape(self.shape)
        self.fs_context["h"] = VZ

    
    #DONE

In [3]:
#Creating a basic model removing channel/hillslope/uplift processes and adding the snapping
model = fst.models.basic_model.drop_processes({"spl","diffusion","uplift", "init_topography"}).update_processes({
    "snast": Snap2SteadySF,
    "topography": Elevation,
    'precip': constant_prec_fastscape_ext.Precipitation,
    'drainage': constant_prec_fastscape_ext.DrainageDischarge,
    
})
model

<xsimlab.Model (14 processes, 10 inputs)>
grid
    shape         [in] ('shape_yx',) nb. of grid nodes in (y, x)
    length        [in] ('shape_yx',) total grid length in (y, x)
boundary
    status        [in] () or ('border',) node status at borders
fs_context
tectonics
surf2erode
init_erosion
flow
precip
    max_rain      [in] maximum rainfall value
    min_rain      [in] minimum rainfall value
drainage
erosion
vmotion
topography
    elevation  [inout] ('y', 'x') surface topography elevation
terrain
snast
    m          [inout] m exp spl
    n          [inout] n exp spl
    K          [inout] ('y', 'x') Erodibility coefficient
    EU         [inout] ('y', 'x') Erosion/Uplift field to target

## Theta scenarios
Run for theta = 0.35 and 0.55


In [5]:
nit = np.arange(500)
## Outputs every XX SHOULD BE 10
nout = nit[::10]

px_res = 30. # pixel resolution in meters
# length of the grid in meters - 15 x 30 km
Lx = 1.5e4 
Ly = 3e4
# Model dimensions
nx = int(Lx/px_res)
ny = int(Ly/px_res)

In [6]:
#Erodibility from paper Bernard et al., 2021
def striped_lithology(nx, ny):
    soft_rock_K = 1e-7 # sedimentary
    hard_rock_K = 5e-8 # crystalline
    K = np.zeros((ny,nx))
    K[:int(ny), int(nx/4):int(nx - (nx/4))] = hard_rock_K
    K[:int(ny), :int(nx/4)] = soft_rock_K
    K[:int(ny), int(nx - (nx/4)):] = soft_rock_K
    return K


### Lithology blobs

In [7]:
def blob_lithology(nx, ny):
    K = DS.diamond_square(shape=(ny,nx), 
                             min_height=5e-8, 
                             max_height=1e-7,
                             roughness=0.7, random_seed = 420)
    return K

In [8]:
from perlin_noise import PerlinNoise
def dense_blob_lithology(nx, ny):    
    noise = PerlinNoise(octaves=7, seed=420)
    pic = [[noise([i/nx, j/ny]) for j in range(nx)] for i in range(ny)]
    K = np.array(pic)
    map_K_arr = (K - K.min()) / (K.max() - K.min())
    map_K_arr = (map_K_arr + 1)*(5e-8)

    return map_K_arr

def homogeneous_lithology(nx, ny):
    # base lithology
    base_litho = 5.4e-8
    K = np.zeros((ny,nx))
    K[:,:] = base_litho
    
    return K

In [29]:
# %create_setup model
# import xsimlab as xs

# General params
## number of iterations
nit = np.arange(500)
## Outputs every XX SHOULD BE 10
nout = nit[::10]

px_res = 30. # pixel resolution in meters
# length of the grid in meters - 15 x 30 km
Lx = 1.5e4 
Ly = 3e4
# Model dimensions
nx = int(Lx/px_res)
ny = int(Ly/px_res)

#make a height map of size nx x ny, with values ranging from 1 to 100, with moderate roughness
diamond = DS.diamond_square(shape=(ny,nx), 
                         min_height=0, 
                         max_height=1,
                         roughness=0.75, random_seed = 420)


#Uplift
EU = np.zeros((ny,nx))+1e-5
#The model
ds_in = xs.create_setup(
    model=model,
    clocks={
        "n_iterations": nit,
        "out": nout
    },
    master_clock='n_iterations',
    input_vars={
        'grid__shape': [ny,nx],
        'grid__length': [Ly,Lx],
        'boundary__status': ["fixed_value", 'fixed_value', 'looped', 'looped'],
        'topography__elevation': diamond,
        'snast__m': 0.45,
        'snast__n': 1,
        'snast__K': litho_diamond, #K,
        'snast__EU': EU,
        'precip': {
            'max_rain' : 5.,
            'min_rain' : 1.
        }
        
    },
    
    output_vars={
        'snast__StStZ': 'out',
        'drainage__flowacc': 'out',
        'precip__precip_rate': 'out'
    }
)


In [30]:
grad_number = 4
file_path = '/exports/csce/datastore/geos/users/s1440040/projects/phd-fastscape/phd-fastscape/precipitation_analysis/models/'


In [31]:
#Running it
zg = zarr.group(f"{file_path}snap2steady_diamond_lithologies_discharge_grad_{grad_number}.zarr", overwrite=True)
with model,xs.monitoring.ProgressBar():
    out_ds = ds_in.xsimlab.run(store = zg)

             0% | initialize 

# Automate running and saving the model outputs

In [11]:
# Automate the snapping to steady state
# %create_setup model
# import xsimlab as xs
file_path = '/exports/csce/datastore/geos/users/s1440040/projects/phd-fastscape/phd-fastscape/precipitation_analysis/models/'

# General params
## number of iterations
nit = np.arange(500)
## Outputs every XX SHOULD BE 10
nout = nit[::10]

px_res = 30. # pixel resolution in meters
# length of the grid in meters - 15 x 30 km
Lx = 1.5e4 
Ly = 3e4
# Model dimensions
nx = int(Lx/px_res)
ny = int(Ly/px_res)

#make a height map of size nx x ny, with values ranging from 1 to 100, with moderate roughness
diamond = DS.diamond_square(shape=(ny,nx), 
                         min_height=0, 
                         max_height=1,
                         roughness=0.75, random_seed = 420)


#Uplift
EU = np.zeros((ny,nx))+1e-5
#The model
K_cases = ['homogeneous_lithology']#['striped_lithology', 'blob_lithology']
theta_values = [0.35, 0.55]

rain_gradients = [0,1,2,3,4,5,6,7,8,9,10]

for theta_val in theta_values:
    for i in range(len(K_cases)):
        if K_cases[i] == 'striped_lithology':
            print('Hi from striped lithology')
            K = striped_lithology(nx, ny)
        elif K_cases[i] == 'dense_blob_lithology':
            print('Hi from dense blob lithology')
            K = dense_blob_lithology(nx, ny)
        elif K_cases[i] == 'blob_lithology':
            print('Hi from blob lithology')
            K = blob_lithology(nx, ny)
        else:
            print('Hi from homogeneous lithology')
            K = homogeneous_lithology(nx, ny)
        for grad in rain_gradients:
            print(f'K case: {K_cases[i]}, gradient: {grad}')
            ds_in = xs.create_setup(
                model=model,
                clocks={
                    "n_iterations": nit,
                    "out": nout
                },
                master_clock='n_iterations',
                input_vars={
                    'grid__shape': [ny,nx],
                    'grid__length': [Ly,Lx],
                    'boundary__status': ["fixed_value", 'fixed_value', 'looped', 'looped'],
                    'topography__elevation': diamond,
                    'snast__m': theta_val,
                    'snast__n': 1,
                    'snast__K': K, #K,
                    'snast__EU': EU,
                    'precip': {
                        'max_rain' : grad + 1.,
                        'min_rain' : 1.
                    }

                },

                output_vars={
                    'snast__StStZ': 'out',
                    'drainage__flowacc': 'out',
                    'precip__precip_rate': 'out'
                }
            )
            #Running it
            zg = zarr.group(f"{file_path}snap2steady_discharge_grad_{grad}_theta_{theta_val}.zarr", overwrite=True)
            with model,xs.monitoring.ProgressBar():
                out_ds = ds_in.xsimlab.run(store = zg)

            lsd.raster_loader.save_raster(
            out_ds["snast__StStZ"].isel({"out": -1}).values[::-1], # if you want to invert the model but keep the dimensions you can also cal that line
            x_min = 0,
            x_max = Lx,
            y_min = Ly, # alternatively you can inverse the two
            y_max = 0, # alternatively you can inverse the two
            res = px_res,
            crs = "EPSG:32635",
            fname = f"{file_path}snap2steady_discharge_grad_{grad}_theta_{theta_val}.tif")



Hi from homogeneous lithology
K case: homogeneous_lithology, gradient: 0


             0% | initialize 

K case: homogeneous_lithology, gradient: 1


             0% | initialize 

K case: homogeneous_lithology, gradient: 2


...

             0% | initialize 

K case: homogeneous_lithology, gradient: 9


             0% | initialize 

K case: homogeneous_lithology, gradient: 10


             0% | initialize 

# Save the output as an lsdtt object

In [32]:

import lsdtopytools as lsd

lsd.raster_loader.save_raster(
#     out_ds["topography__elevation"].isel({"out": -1}).values, # The raster as the last output from the model
    out_ds["snast__StStZ"].isel({"out": -1}).values[::-1], # if you want to invert the model but keep the dimensions you can also cal that line
    x_min = 0,
    x_max = Lx,
    y_min = Ly, # alternatively you can inverse the two
    y_max = 0, # alternatively you can inverse the two
    res = px_res,
    crs = "EPSG:32635",
    fname = f"{file_path}snap2steady_diamond_lithologies_discharge_grad_{grad_number}.tif"
)

In [33]:
# Save also the rainfall raster
lsd.raster_loader.save_raster(
#     out_ds["topography__elevation"].isel({"out": -1}).values, # The raster as the last output from the model
    out_ds["precip__precip_rate"].isel({"out": -1}).values[::-1], # if you want to invert the model but keep the dimensions you can also cal that line
    x_min = 0,
    x_max = Lx,
    y_min = Ly, # alternatively you can inverse the two
    y_max = 0, # alternatively you can inverse the two
    res = px_res,
    crs = "EPSG:32635",
    fname = f"{file_path}precip_snap2steady_diamond_lithologies_discharge_grad_{grad_number}.tif"
)

# Visualise the result

In [12]:
from ipyfastscape import TopoViz3d


app = TopoViz3d(out_ds, canvas_height=600, time_dim="out",elevation_var = "snast__StStZ" )

app.show()

Output(layout=Layout(height='640px'))